# Post procesing of the json-files
- Remove articles in false order
- Linking cross references

In [ ]:
from tqdm.notebook import tqdm
import re
import json
import bisect

edition_test = 'cross_test'
e1 = 'e1'
e2 = 'e2'

In [ ]:
# Get entries from json file
def get_entries(edition_name: str) -> list[dict]:
    with open(f"{edition_name}.json", 'r', encoding='utf-8') as infile:
        entries = json.loads(infile.read())
    return entries

# Write entries to json file
def dump_entries(entries: list[dict], edition_name: str) -> None:
    with open(f"{edition_name}.json", 'w', encoding='utf-8') as outfile:
        json.dump(entries, outfile, ensure_ascii=False, indent=4)

# Get reference word (what is after 'Se ')
def get_ref_word(text: str) -> str:
    match = re.search(r'Se\s+([^.]+)\.', text)
    if match:
        return match.group(1).strip()
    return ""

# Find index of headword
def binary_search(arr, target):
    index = bisect.bisect_left(arr, target)
    if index != len(arr) and arr[index] == target:
        return index
    return -1

# Cross reference linking logic
def cross_link(edition_name: str) -> None:
    entries = get_entries(edition_name)
    headwords = [entry['headword'] for entry in tqdm(entries, desc="Retrieving Headwords")]

    # Assign cross references
    for entry in tqdm(entries, desc="Finding cross references"):
        if entry['is_cross_ref']:
            ref_word = get_ref_word(entry['text'])
            idx = binary_search(headwords, ref_word)
            if idx != -1:
                entry['cross_id'] = entries[idx]['entryid']
                continue
        else: # REMOVE IF ENTRIES ALREADY HAVE KEY 'CROSS_ID'
            entry['cross_id'] = ''
    
    dump_entries(entries, edition_name)

In [ ]:
cross_link(edition_test)